In [1]:
pip install dune-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 2.0.0
    Uninstalling wrapt-2.0.0:
      Successfully uninstalled wrapt-2.0.0
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.13.1
    Uninstalling aiohttp-3.13.1:
      Successfully uninstalled aiohttp-3.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4

In [51]:
from dune_client.client import DuneClient
from dune_client.query import QueryBase, QueryParameter
from dune_client.types import ParameterType
from dune_client.models import ExecutionState
from datetime import datetime, timedelta, timedelta, timezone
import time
import pandas as pd
API_KEY = "ESWNsFNNW9nLwn5snL7RG79sj20QuQbz" #Ohm
POOL_ADDRESS = "4e68ccd3e89f51c3074ca5072bbac773960dfa36"
dune = DuneClient(API_KEY)

In [53]:
#SWAPS -CHUNK (NO NEED IF UNLIMIT TOKEN PER API CALL)
QUERY_ID = 6055461

start_full = datetime(2025, 9, 1, 0, 0,  tzinfo=timezone.utc)
end_full   = datetime(2025,10, 1, 0, 0, tzinfo=timezone.utc)

dfs = []

cursor = start_full
while cursor < end_full:
    next_cut = min(cursor + timedelta(days=1), end_full)

    #parameter list
    params_list = [
        QueryParameter(
            name="address_parameter",
            parameter_type=ParameterType.TEXT,
            value=POOL_ADDRESS,
        ),
        QueryParameter(
            name="start_date_parameter",
            parameter_type=ParameterType.TEXT,
            value=cursor,
        ),
        QueryParameter(
            name="end_date_parameter",
            parameter_type=ParameterType.TEXT,
            value=next_cut,
        ),
    ]

    q = QueryBase(
        query_id=QUERY_ID,
        params=params_list,
    )

    exec_resp = dune.execute_query(query=q)
    execution_id = exec_resp.execution_id
    print(f"[{cursor} → {next_cut}] execution_id:", execution_id)

    terminal_states = {
      ExecutionState.COMPLETED,
      ExecutionState.PARTIAL,
      ExecutionState.FAILED,
      ExecutionState.CANCELLED,
      ExecutionState.EXPIRED,
    }

    while True:
        status = dune.get_execution_status(execution_id)
        state = status.state
        print(f"[{cursor} → {next_cut}] state:", state)

        if state in terminal_states:
            break

        time.sleep(10)

    if state != ExecutionState.COMPLETED:
        print(f"[{cursor} → {next_cut}] query did not complete. final state = {state}")
        cursor = next_cut
        continue

    res = dune.get_execution_results(execution_id)

    #SDK versions differ in shape. Try modern first.
    rows = getattr(res, "result_rows", None)
    if rows is None:
        # legacy attribute path
        rows = res.result.rows

    df_chunk = pd.DataFrame(rows)
    df_chunk["slice_start"] = cursor
    df_chunk["slice_end"] = next_cut

    dfs.append(df_chunk)

    cursor = next_cut

#combine and export
if len(dfs) == 0:
    print("No data collected")
else:
    df_all = pd.concat(dfs, ignore_index=True)

    out_path = "swaps_2025_09_01_to_2025_10_01_eth_usdt_0p3.csv"
    df_all.to_csv(out_path, index=False)
    print("Wrote:", out_path)


[2025-09-01 00:00:00+00:00 → 2025-09-02 00:00:00+00:00] execution_id: 01K8JX95RE3CGVQB974K2V0QTW
[2025-09-01 00:00:00+00:00 → 2025-09-02 00:00:00+00:00] state: ExecutionState.PENDING
[2025-09-01 00:00:00+00:00 → 2025-09-02 00:00:00+00:00] state: ExecutionState.COMPLETED
[2025-09-02 00:00:00+00:00 → 2025-09-03 00:00:00+00:00] execution_id: 01K8JX9GCXH82R3KFD2PTPXRVP
[2025-09-02 00:00:00+00:00 → 2025-09-03 00:00:00+00:00] state: ExecutionState.PENDING
[2025-09-02 00:00:00+00:00 → 2025-09-03 00:00:00+00:00] state: ExecutionState.COMPLETED
[2025-09-03 00:00:00+00:00 → 2025-09-04 00:00:00+00:00] execution_id: 01K8JX9V0RJKMS9VWYDHMC6NM2
[2025-09-03 00:00:00+00:00 → 2025-09-04 00:00:00+00:00] state: ExecutionState.PENDING
[2025-09-03 00:00:00+00:00 → 2025-09-04 00:00:00+00:00] state: ExecutionState.COMPLETED
[2025-09-04 00:00:00+00:00 → 2025-09-05 00:00:00+00:00] execution_id: 01K8JXA5HH54AYMGYDA2E7Y7XE
[2025-09-04 00:00:00+00:00 → 2025-09-05 00:00:00+00:00] state: ExecutionState.PENDING
[202

In [55]:
#MINT

QUERY_ID = 6055478

start_full = datetime(2025, 9, 1, 0, 0, tzinfo=timezone.utc)
end_full   = datetime(2025,10, 1, 0, 0, tzinfo=timezone.utc)

params_list = [
    QueryParameter(
        name="address_parameter",
        parameter_type=ParameterType.TEXT,
        value=POOL_ADDRESS,
    ),
    QueryParameter(
        name="start_date_parameter",
        parameter_type=ParameterType.TEXT,
        value=start_full.strftime("%Y-%m-%d %H:%M:%S UTC"),
    ),
    QueryParameter(
        name="end_date_parameter",
        parameter_type=ParameterType.TEXT,
        value=end_full.strftime("%Y-%m-%d %H:%M:%S UTC"),
    ),
]

q = QueryBase(
    query_id=QUERY_ID,
    params=params_list,
)

exec_resp = dune.execute_query(query=q)
execution_id = exec_resp.execution_id
print("execution_id:", execution_id)

terminal_states = {
    ExecutionState.COMPLETED,
    ExecutionState.PARTIAL,
    ExecutionState.FAILED,
    ExecutionState.CANCELLED,
    ExecutionState.EXPIRED,
}

while True:
    status = dune.get_execution_status(execution_id)
    state = status.state
    print("state:", state)
    if state in terminal_states:
        break
    time.sleep(2)

if state not in (ExecutionState.COMPLETED, ExecutionState.PARTIAL):
    raise RuntimeError(f"Query failed. Final state = {state}")

res = dune.get_execution_results(execution_id)

rows = getattr(res, "result_rows", None)
if rows is None:
    rows = res.result.rows

df_all = pd.DataFrame(rows)

out_path = "mints_2025_09_01_to_2025_10_01_eth_usdt_0p3.csv"
df_all.to_csv(out_path, index=False)
print("Wrote:", out_path)


execution_id: 01K8JY3PMQFZ6HMFTE17Q37Y14
state: ExecutionState.PENDING
state: ExecutionState.EXECUTING
state: ExecutionState.EXECUTING
state: ExecutionState.COMPLETED
Wrote: mints_2025_09_01_to_2025_10_01_eth_usdt_0p3.csv


In [57]:
#BURN

QUERY_ID = 6055486

start_full = datetime(2025, 9, 1, 0, 0, tzinfo=timezone.utc)
end_full   = datetime(2025,10, 1, 0, 0, tzinfo=timezone.utc)


params_list = [
    QueryParameter(
        name="address_parameter",
        parameter_type=ParameterType.TEXT,
        value=POOL_ADDRESS,
    ),
    QueryParameter(
        name="start_date_parameter",
        parameter_type=ParameterType.TEXT,
        value=start_full.strftime("%Y-%m-%d %H:%M:%S UTC"),
    ),
    QueryParameter(
        name="end_date_parameter",
        parameter_type=ParameterType.TEXT,
        value=end_full.strftime("%Y-%m-%d %H:%M:%S UTC"),
    ),
]

q = QueryBase(
    query_id=QUERY_ID,
    params=params_list,
)

exec_resp = dune.execute_query(query=q)
execution_id = exec_resp.execution_id
print("execution_id:", execution_id)

terminal_states = {
    ExecutionState.COMPLETED,
    ExecutionState.PARTIAL,
    ExecutionState.FAILED,
    ExecutionState.CANCELLED,
    ExecutionState.EXPIRED,
}

while True:
    status = dune.get_execution_status(execution_id)
    state = status.state
    print("state:", state)
    if state in terminal_states:
        break
    time.sleep(2)

if state not in (ExecutionState.COMPLETED, ExecutionState.PARTIAL):
    raise RuntimeError(f"Query failed. Final state = {state}")

res = dune.get_execution_results(execution_id)

rows = getattr(res, "result_rows", None)
if rows is None:
    rows = res.result.rows

df_all = pd.DataFrame(rows)

out_path = "burns_2025_09_01_to_2025_10_01_eth_usdt_0p3.csv"
df_all.to_csv(out_path, index=False)
print("Wrote:", out_path)

execution_id: 01K8JY8QDM077HEG8GHXBJRGFA
state: ExecutionState.PENDING
state: ExecutionState.EXECUTING
state: ExecutionState.COMPLETED
Wrote: burns_2025_09_01_to_2025_10_01_eth_usdt_0p3.csv


In [58]:
#TOKEN METADATA , POOL CONFIG

data = [
    {
        "contract_address": "0xdac17f958d2ee523a2206206994597c13d831ec7",
        "symbol": "USDT",
        "decimals": 6,
    },
    {
        "contract_address": "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2",
        "symbol": "WETH",
        "decimals": 18,
    },
]

df_tokens = pd.DataFrame(data)
out_path = "token_metadata_eth_usdt_0p3.csv"
df_tokens.to_csv(out_path, index=False)

import pandas as pd

data = [
    {
        "pool": "0x4e68ccd3e89f51c3074ca5072bbac773960dfa36",
        "token0": "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2",
        "token1": "0xdac17f958d2ee523a2206206994597c13d831ec7",
        "fee": 3000,
        "tickSpacing": 60,
    }
]

df_pool = pd.DataFrame(data)
out_path = "pool_config_eth_usdt_0p3.csv"
df_pool.to_csv(out_path, index=False)
